In [ ]:
import os, sys
import shutil
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Para no tener problemas de relative imports with packages
from utils import util

In [ ]:
# Functions
def load_raw_audios(dataframe, source, dest):
    ''' Searches in the source dir, the audio files present in the dataframe and copies them to the dest dir.'''
    for _, row in dataframe.iterrows():
        primary_label = row['primary_label']
        filename = row['filename']
        
        source_path = os.path.join(source, primary_label, filename)
        destination_path = os.path.join(dest, filename)

        if os.path.exists(source_path):
            shutil.copy2(source_path, destination_path)

In [ ]:
df = pd.read_csv(os.path.join('..', 'database', 'meta', 'final_species.csv'))

repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
os.chdir(repo_root)

audios_dir = os.path.join('..', 'data', 'birdclef-2021', 'train_short_audio')
oggs_dir = os.path.join('..', 'database', 'assets', 'raw')

In [ ]:
# Reset, then load the raw audio files
print("Resetting raw audio files...")
util.clean_dir(oggs_dir)

print("Loading raw audio files...")
load_raw_audios(df, audios_dir, oggs_dir)

Resetting raw audio files...
Loading raw audio files...
